## DS4200 Group Project: Comparing Tesla Stock and Weather Data
Group members: Emma Rabbath, Beth Farr, Anya Solanki, Charlotte Thunen


In [3]:
import pandas as pd
import os

In [4]:
## Load all data into separate weather and tesla dfs
# Weather Data
# File path
file_path = "GroupProject/Austin_Weather_Data/"
# List of files
files = [
    "Austin_Texas_2014-09-15_2017-06-09.csv",
    "Austin_Texas_2017-06-10_2019-03-24.csv",
    "Austin_Texas_2019-03-25_2021-12-18.csv",
    "Austin_Texas_2021-12-19_2024-09-13.csv"
]

# Full file paths
file_paths = [os.path.join(file_path, file) for file in files]

# Combine all files into one df
df_list = [pd.read_csv(file) for file in file_paths]
weather_df = pd.concat(df_list, ignore_index=True)

# Tesla Data
tesla_df = pd.read_csv("GroupProject/Tesla_Stock_Data/Tesla_Stock_2014-09-15_2024_09_13.csv")


In [5]:
## Cleaning data
# Convert weather_df "datetime" to datetime object
weather_df['datetime'] = pd.to_datetime(weather_df['datetime'])

# Convert tesla_df "date" to datetime object
tesla_df['datetime'] = pd.to_datetime(tesla_df['Date'], format='%m/%d/%Y')

# Remove special symbols from each column in tesla_df
tesla_df['Close/Last'] = tesla_df['Close/Last'].replace({'\$': '', ',': ''}, regex=True).astype(float)
tesla_df['Open'] = tesla_df['Open'].replace({'\$': '', ',': ''}, regex=True).astype(float)
tesla_df['High'] = tesla_df['High'].replace({'\$': '', ',': ''}, regex=True).astype(float)
tesla_df['Low'] = tesla_df['Low'].replace({'\$': '', ',': ''}, regex=True).astype(float)

# Set datetime as index to merge the dataframes
weather_df.set_index('datetime', inplace=True)
tesla_df.set_index('datetime', inplace=True)

# Combine all data into one df
combined_df = weather_df.join(tesla_df, how='inner')

In [14]:
combined_df.columns

Index(['name', 'tempmax', 'tempmin', 'temp', 'feelslikemax', 'feelslikemin',
       'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'precipcover',
       'preciptype', 'snow', 'snowdepth', 'windgust', 'windspeed', 'winddir',
       'sealevelpressure', 'cloudcover', 'visibility', 'solarradiation',
       'solarenergy', 'uvindex', 'severerisk', 'sunrise', 'sunset',
       'moonphase', 'conditions', 'description', 'icon', 'stations', 'Date',
       'Close/Last', 'Volume', 'Open', 'High', 'Low'],
      dtype='object')

In [16]:
combined_df.head()

,name,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,...,conditions,description,icon,stations,Date,Close/Last,Volume,Open,High,Low
datetime,,,,,,,,,,,,,,,,,,,,,
2014-09-15,"Austin, TX",86.4,69.4,76.8,90.6,69.4,78.4,68.0,75.6,0.000,...,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"72253912979,KATT,KAUS,72064800230,KEDC,7225401...",09/15/2014,16.9240,246507477,18.2913,18.2933,16.6087
2014-09-16,"Austin, TX",93.4,74.5,81.3,99.6,74.5,83.9,71.9,75.4,0.018,...,"Rain, Partially cloudy",Becoming cloudy in the afternoon with late aft...,rain,"KATT,KAUS,72064800230,KEDC,72254013904,7225441...",09/16/2014,17.3827,124461748,17.0100,17.4973,16.8280
2014-09-17,"Austin, TX",88.2,74.9,80.8,96.6,74.9,84.0,73.4,79.5,0.004,...,"Rain, Partially cloudy",Partly cloudy throughout the day with afternoo...,rain,"KATT,KAUS,72064800230,KEDC,72254013904,7225441...",09/17/2014,17.4253,77594871,17.4940,17.6467,17.3000
2014-09-18,"Austin, TX",88.7,71.6,77.6,95.2,71.6,79.2,72.5,86.0,3.253,...,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,"KATT,KAUS,72064800230,KEDC,72254013904,7225441...",09/18/2014,17.5880,55342127,17.5573,17.7067,17.4880
2014-09-19,"Austin, TX",84.1,74.0,77.9,90.7,74.0,79.7,73.5,87.2,0.240,...,"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...,rain,"KATT,KAUS,72064800230,KEDC,72254013904,7225441...",09/19/2014,17.2880,102014574,17.1993,17.4287,17.0180
